# Save energy terms along same contours as transport so they can be compared

Did this for a lot of quantities so they could be compared along contours

In [1]:
%matplotlib inline

import cosima_cookbook as cc
import matplotlib.pyplot as plt
import numpy as np
import netCDF4 as nc
import xarray as xr
import cftime
import glob
import dask
import os
import datetime
import matplotlib.colors as col
import dask.array as dsa
from cosima_cookbook import distributed as ccd
import cartopy.crs as ccrs
import cmocean as cm
from scipy.ndimage.filters import uniform_filter1d
import matplotlib as mpl
from collections import OrderedDict


import logging
logging.captureWarnings(True)
logging.getLogger('py.warnings').setLevel(logging.ERROR)

from dask.distributed import Client

In [2]:
client = Client()
client

Client Scheduler: tcp://127.0.0.1:42705 Dashboard: /proxy/44783/status,Cluster Workers: 4 Cores: 16 Memory: 64.00 GiB


In [3]:
EKE = xr.open_dataset('/g/data/x77/cy8964/Post_Process/New_SO/EKE_10yr.nc').EKE
MKE = xr.open_dataset('/g/data/x77/cy8964/Post_Process/New_SO/MKE_10yr.nc').MKE
Reynolds = xr.open_dataset('/g/data/x77/cy8964/Post_Process/New_SO/Reynolds_10yr_fulldepth.nc').Reynolds
Form_stress = xr.open_dataset('/g/data/x77/cy8964/Post_Process/New_SO/Form_stress_10yr_fulldepth.nc').Form_stress

uh_bar = xr.open_dataset('/g/data/x77/cy8964/Post_Process/New_SO/uh_bar_10yr.nc').uh_bar
vh_bar = xr.open_dataset('/g/data/x77/cy8964/Post_Process/New_SO/vh_bar_10yr.nc').vh_bar
h_bar = xr.open_dataset('/g/data/x77/cy8964/Post_Process/New_SO/h_bar_10yr.nc').h_bar
u_hat = xr.open_dataset('/g/data/x77/cy8964/Post_Process/New_SO/u_hat_10yr.nc').u_hat
v_hat = xr.open_dataset('/g/data/x77/cy8964/Post_Process/New_SO/v_hat_10yr.nc').v_hat

In [3]:
session = cc.database.create_session('/g/data/ik11/databases/cosima_master.db')

In [4]:
# This example script just calculates transports for a single year. 
# You'll need to run it multiple times (or set up loop over years)

year = '2170'
expt = '01deg_jra55v13_ryf9091'

start_time= year + '-01-01'
end_time= year + '-12-31'

# reference density value:
rho_0 = 1035.0
# Note: change this range, so it matches the size of your contour arrays:

## FULL SO ##
lat_range = slice(-70,-34.99)
lat_range_big =  slice(-70.05,-34.90)



In [5]:
contour_dict = OrderedDict([
    ('A',  {'SSH':-0.1,'lat_range':slice(-60,-34.99),'lat_range_big':slice(-60.05,-34.90),'slope_path':'SO_slope_contour_-0.1m_SSH.npz','shifted':False}),
    ('B',  {'SSH':-0.2,'lat_range':slice(-60,-34.99),'lat_range_big':slice(-60.05,-34.90),'slope_path':'SO_slope_contour_-0.2m_SSH.npz','shifted':False}),
    ('C',  {'SSH':-0.3,'lat_range':slice(-60,-34.99),'lat_range_big':slice(-60.05,-34.90),'slope_path':'SO_slope_contour_-0.3m_SSH.npz','shifted':False}),
    ('D',  {'SSH':-0.4,'lat_range':slice(-60,-34.99),'lat_range_big':slice(-60.05,-34.90),'slope_path':'SO_slope_contour_-0.4m_SSH.npz','shifted':False}),
    ('E',  {'SSH':-0.5,'lat_range':slice(-60,-34.99),'lat_range_big':slice(-60.05,-34.90),'slope_path':'SO_slope_contour_-0.5m_SSH.npz','shifted':False}),
    ('F',  {'SSH':-0.6, 'lat_range':slice(-60,-39.98),'lat_range_big':slice(-60.05,-39.90),'slope_path':'SO_slope_contour_-0.6m_SSH.npz','shifted':False}),
    ('G',  {'SSH':-0.7, 'lat_range':slice(-60,-39.98),'lat_range_big':slice(-60.05,-39.90),'slope_path':'SO_slope_contour_-0.7m_SSH.npz','shifted':False}),
    ('H',  {'SSH':-0.8, 'lat_range':slice(-62.91,-45), 'lat_range_big':slice(-62.96,-44.90),'slope_path':'SO_slope_contour_-0.8m_SSH.npz','shifted':False}),
    ('I',  {'SSH':-0.9, 'lat_range':slice(-62.91,-45), 'lat_range_big':slice(-62.96,-44.90),'slope_path':'SO_slope_contour_-0.9m_SSH.npz','shifted':False}),
    ('J',  {'SSH':-1.0, 'lat_range':slice(-62.91,-45), 'lat_range_big':slice(-62.96,-44.90),'slope_path':'SO_slope_contour_-1.0m_SSH.npz','shifted':False}),
    ('K',  {'SSH':-1.1, 'lat_range':slice(-64.99,-47), 'lat_range_big':slice(-65.02,-46.93),'slope_path':'SO_slope_contour_-1.1m_SSH.npz','shifted':False}),
    ('L',  {'SSH':-1.2, 'lat_range':slice(-64.99,-47), 'lat_range_big':slice(-65.02,-46.93),'slope_path':'SO_slope_contour_-1.2m_SSH.npz','shifted':False}),
    ('M',  {'SSH':-1.3, 'lat_range':slice(-70,-47), 'lat_range_big':slice(-70.05,-46.93),'slope_path':'SO_slope_contour_-1.3m_SSH.npz','shifted':False}),
    ('N',  {'SSH':-1.4, 'lat_range':slice(-70,-47), 'lat_range_big':slice(-70.05,-46.93),'slope_path':'SO_slope_contour_-1.4m_SSH.npz','shifted':False}),
    ('O',  {'SSH':-1.5, 'lat_range':slice(-70,-47), 'lat_range_big':slice(-70.05,-46.93),'slope_path':'SO_slope_contour_-1.5m_SSH.npz','shifted':True}),
    ('P',  {'SSH':-0.15,'lat_range':slice(-60,-34.99),'lat_range_big':slice(-60.05,-34.90),'slope_path':'SO_slope_contour_-0.15m_SSH.npz','shifted':False}),
    ('Q',  {'SSH':-0.25,'lat_range':slice(-60,-34.99),'lat_range_big':slice(-60.05,-34.90),'slope_path':'SO_slope_contour_-0.25m_SSH.npz','shifted':False}),
    ('R',  {'SSH':-0.35,'lat_range':slice(-60,-34.99),'lat_range_big':slice(-60.05,-34.90),'slope_path':'SO_slope_contour_-0.35m_SSH.npz','shifted':False}),
    ('S',  {'SSH':-0.45,'lat_range':slice(-60,-34.99),'lat_range_big':slice(-60.05,-34.90),'slope_path':'SO_slope_contour_-0.45m_SSH.npz','shifted':False}),
    ('T',  {'SSH':-0.55,'lat_range':slice(-60,-34.99),'lat_range_big':slice(-60.05,-34.90),'slope_path':'SO_slope_contour_-0.55m_SSH.npz','shifted':False}),
    ('U',  {'SSH':-0.65, 'lat_range':slice(-60,-39.98),'lat_range_big':slice(-60.05,-39.90),'slope_path':'SO_slope_contour_-0.65m_SSH.npz','shifted':False}),
    ('V',  {'SSH':-0.75, 'lat_range':slice(-61,-39.98),'lat_range_big':slice(-61.05,-39.90),'slope_path':'SO_slope_contour_-0.75m_SSH.npz','shifted':False}),
    ('W',  {'SSH':-0.85,'lat_range':slice(-62.91,-45), 'lat_range_big':slice(-62.96,-44.90),'slope_path':'SO_slope_contour_-0.85m_SSH.npz','shifted':False}),
    ('X',  {'SSH':-0.95, 'lat_range':slice(-62.91,-45), 'lat_range_big':slice(-62.96,-44.90),'slope_path':'SO_slope_contour_-0.95m_SSH.npz','shifted':False}),
    ('Y',  {'SSH':-1.05, 'lat_range':slice(-62.91,-45), 'lat_range_big':slice(-62.96,-44.90),'slope_path':'SO_slope_contour_-1.05m_SSH.npz','shifted':False}),
    ('Z',  {'SSH':-1.15, 'lat_range':slice(-64.99,-47), 'lat_range_big':slice(-65.02,-46.93),'slope_path':'SO_slope_contour_-1.15m_SSH.npz','shifted':False}),
    ('Z1',  {'SSH':-1.25, 'lat_range':slice(-70,-47), 'lat_range_big':slice(-70.05,-46.93),'slope_path':'SO_slope_contour_-1.25m_SSH.npz','shifted':False}),
    ('Z2',  {'SSH':-1.35, 'lat_range':slice(-70,-47), 'lat_range_big':slice(-70.05,-46.93),'slope_path':'SO_slope_contour_-1.35m_SSH.npz','shifted':False}),
    ('Z3',  {'SSH':-1.45, 'lat_range':slice(-70,-47), 'lat_range_big':slice(-70.05,-46.93),'slope_path':'SO_slope_contour_-1.45m_SSH.npz','shifted':True}),
    ('Z4',  {'SSH':-1.55, 'lat_range':slice(-70,-47), 'lat_range_big':slice(-70.05,-46.93),'slope_path':'SO_slope_contour_-1.55m_SSH_A.npz','shifted':True}),
    ('Z5',  {'SSH':-1.55, 'lat_range':slice(-70,-47), 'lat_range_big':slice(-70.05,-46.93),'slope_path':'SO_slope_contour_-1.55m_SSH_B.npz','shifted':True}),
    ('Z6',  {'SSH':-1.6, 'lat_range':slice(-70,-47), 'lat_range_big':slice(-70.05,-46.93),'slope_path':'SO_slope_contour_-1.6m_SSH_A.npz','shifted':True}),
    ('Z7',  {'SSH':-1.6, 'lat_range':slice(-70,-47), 'lat_range_big':slice(-70.05,-46.93),'slope_path':'SO_slope_contour_-1.6m_SSH_B.npz','shifted':True}),
    ('Z8',  {'SSH':-1.65, 'lat_range':slice(-70,-47), 'lat_range_big':slice(-70.05,-46.93),'slope_path':'SO_slope_contour_-1.65m_SSH_A.npz','shifted':True}),
    ('Z9',  {'SSH':-1.65, 'lat_range':slice(-70,-47), 'lat_range_big':slice(-70.05,-46.93),'slope_path':'SO_slope_contour_-1.65m_SSH_B.npz','shifted':True}),
])

shifted is because for some contours I had to initially shift the grid so that the contour was continuous, as it bended back at the boundary.

# save form stress EKE reynolds on contours

In [6]:
# EKE = xr.open_dataset('/g/data/x77/cy8964/Post_Process/New_SO/EKE_10yr.nc').EKE
# Reynolds= xr.open_dataset('/g/data/x77/cy8964/Post_Process/New_SO/Reynolds_10yr_fulldepth.nc').Reynolds
# Form_stress= xr.open_dataset('/g/data/x77/cy8964/Post_Process/New_SO/Form_stress_10yr_fulldepth.nc').Form_stress
EKE = xr.open_dataset('/g/data/x77/cy8964/Post_Process/New_SO/EKE_10yr_smoothed_80km.nc').EKE
#Reynolds= xr.open_dataset('/g/data/x77/cy8964/Post_Process/New_SO/Reynolds_1yr_fulldepth.nc').Reynolds
Form_stress= xr.open_dataset('/g/data/x77/cy8964/Post_Process/New_SO/Form_stress_10yr_smoothed_80km.nc').Form_stress
EPE = xr.open_dataset('/g/data/x77/cy8964/Post_Process/New_SO/EPE_10yr_smoothed_80km_fulldepth.nc').EPE


for j in np.arange(35):
    j_contour = np.array(['A','P','B','Q','C','R','D','S','E','T','F','U','G','V','H','W','I','X','J','Y',
                              'K','Z','L','Z1','M','Z2','N','Z3','O','Z4','Z5','Z6','Z7','Z8','Z9'])[j]
#    j_contour = np.array(['O','P','Q','R','S','T','U','V','W','X','Y','Z','Z1','Z2','Z3','Z4','Z5','Z6','Z7','Z8','Z9'])[j]
    print(j_contour)
    lat_range = contour_dict[j_contour]['lat_range']
    lat_range_big = contour_dict[j_contour]['lat_range_big']
    SSH = contour_dict[j_contour]['SSH']
    
    salt = Form_stress.sel(yt_ocean = lat_range)
    if contour_dict[j_contour]['shifted']==True:
        salt = salt.roll(xt_ocean = 800)
        salt.coords['xt_ocean']=np.linspace(0.05,359.95,3600)

    outfile = '/g/data/x77/cy8964/Post_Process/'+contour_dict[j_contour]['slope_path']
    data = np.load(outfile)
    mask_y_transport = data['mask_y_transport']
    mask_x_transport = data['mask_x_transport']
    mask_y_transport_numbered = data['mask_y_transport_numbered']
    mask_x_transport_numbered = data['mask_x_transport_numbered']

    if contour_dict[j_contour]['shifted']==True:
        yt_ocean = cc.querying.getvar(expt,'yt_ocean',session,n=1)
        yt_ocean = yt_ocean.sel(yt_ocean=lat_range)
        yu_ocean = cc.querying.getvar(expt,'yu_ocean',session,n=1)
        yu_ocean = yu_ocean.sel(yu_ocean=lat_range)

        xt_ocean = cc.querying.getvar(expt, 'xt_ocean', session, n=1)
        xt_ocean = xt_ocean.roll(xt_ocean = 800)
        xt_ocean.coords['xt_ocean']=np.linspace(0.05,359.95,3600)
        xt_ocean = xt_ocean*0+xt_ocean.xt_ocean.values

        xu_ocean = cc.querying.getvar(expt, 'xu_ocean', session, n=1)
        xu_ocean = xu_ocean.roll(xu_ocean = 800)
        xu_ocean.coords['xu_ocean']=np.linspace(0.1,360,3600)
        xu_ocean = xu_ocean*0+xu_ocean.xu_ocean.values
        
        dyt = cc.querying.getvar(expt, 'dyt',session, n=1, ncfile = 'ocean_grid.nc')
        dyt = dyt.roll(xt_ocean = 800)
        dyt.coords['xt_ocean']=np.linspace(0.05,359.95,3600)

        dxu = cc.querying.getvar(expt, 'dxu',session, n=1, ncfile = 'ocean_grid.nc')
        dxu = dxu.roll(xu_ocean = 800)
        dxu.coords['xu_ocean']=np.linspace(0.1,360,3600)
        
        # select latitude range:
        dxu = dxu.sel(yu_ocean=lat_range)
        dyt = dyt.sel(yt_ocean=lat_range)

    elif contour_dict[j_contour]['shifted']==False:
        yt_ocean = cc.querying.getvar(expt, 'yt_ocean', session, n=1)
        yt_ocean = yt_ocean.sel(yt_ocean=lat_range)
        yu_ocean = cc.querying.getvar(expt, 'yu_ocean', session, n=1)
        yu_ocean = yu_ocean.sel(yu_ocean=lat_range)

        xt_ocean = cc.querying.getvar(expt, 'xt_ocean', session, n=1)
        xu_ocean = cc.querying.getvar(expt, 'xu_ocean', session, n=1)

        dyt = cc.querying.getvar(expt,'dyt',session,ncfile = 'ocean_grid.nc',n=1)
        dyt = dyt.sel(yt_ocean = lat_range)
        dxt = cc.querying.getvar(expt,'dxt',session,ncfile = 'ocean_grid.nc',n=1)
        dxt = dxt.sel(yt_ocean = lat_range)

        dxu = cc.querying.getvar(expt,'dxu',session,ncfile = 'ocean_grid.nc',n=1)
        dxu = dxu.sel(yu_ocean = lat_range)

    dyt = dyt.reset_coords().dyt # remove geolon_t/geolat_t coordinates
    dxu = dxu.reset_coords().dxu # remove geolon_t/geolat_t coordinates
    dxu.coords['xu_ocean'] = xt_ocean.values
    dxu = dxu.rename({'yu_ocean':'y_ocean', 'xu_ocean':'x_ocean'}) 
    dyt.coords['xt_ocean'] = xu_ocean.values
    dyt = dyt.rename({'yt_ocean':'y_ocean','xt_ocean':'x_ocean'})

    # Convert contour masks to data arrays, so we can multiply them later.
    # We need to ensure the lat lon coordinates correspond to the actual data location:
    #       The y masks are used for vhrho, so like vhrho this should have dimensions (yu_ocean, xt_ocean).
    #       The x masks are used for uhrho, so like uhrho this should have dimensions (yt_ocean, xu_ocean).
    #       However the actual name will always be simply y_ocean/x_ocean irrespective of the variable
    #       to make concatenation of transports in both direction and sorting possible.

    mask_x_transport = xr.DataArray(mask_x_transport, coords = [('y_ocean', yt_ocean), ('x_ocean', xu_ocean)])
    mask_y_transport = xr.DataArray(mask_y_transport, coords = [('y_ocean', yu_ocean), ('x_ocean', xt_ocean)])
    mask_x_transport_numbered = xr.DataArray(mask_x_transport_numbered, coords = [('y_ocean', yt_ocean), ('x_ocean', xu_ocean)])
    mask_y_transport_numbered = xr.DataArray(mask_y_transport_numbered, coords = [('y_ocean', yu_ocean), ('x_ocean', xt_ocean)])

    mask_x_numbered_1d = mask_x_transport_numbered.stack(contour_index = ['y_ocean', 'x_ocean'])
    mask_x_numbered_1d = mask_x_numbered_1d.where(mask_x_numbered_1d > 0, drop = True)
    mask_y_numbered_1d = mask_y_transport_numbered.stack(contour_index = ['y_ocean', 'x_ocean'])
    mask_y_numbered_1d = mask_y_numbered_1d.where(mask_y_numbered_1d > 0, drop = True)
    contour_ordering = xr.concat((mask_x_numbered_1d,mask_y_numbered_1d), dim = 'contour_index')
    contour_ordering = contour_ordering.sortby(contour_ordering)
    contour_index_array = np.arange(1,len(contour_ordering)+1)

    #inititalise empty array
    salt_along_contour = xr.DataArray(np.zeros((len(salt.isopycnal_bins),len(contour_index_array))),
                                      coords = [salt.isopycnal_bins, contour_index_array],
                                      dims = ['isopycnal_bins', 'contour_index'],
                                      name = 'salt_along_contour')

    salt_i = salt.rename({'yt_ocean':'y_ocean', 'xt_ocean':'x_ocean'}) 

    # Note that this interpolation does not work as generically as e.g. salt.interp(),
    #    but it is much faster and doesn't require removing chunking (which also slow things down).
    # Be careful that your latitude range extends at least one point either direction beyond your contour.
    # If your domain is not the full longitude range, you will need to adapt this, so you have the correct interpolation
    #    only the edges of your domain (it assumes it is reentrant).
    # Need to overwrite coords, so these two variables can be added together:
    salt_w = salt_i.copy()
    salt_w.coords['x_ocean'] = xu_ocean.values
    salt_e = salt_i.roll(x_ocean=-1)
    salt_e.coords['x_ocean'] = xu_ocean.values
    # salt_xgrid will be on the uhrho grid:
    salt_xgrid = (salt_e + salt_w)/2

    salt_s = salt_i.copy()
    salt_s.coords['y_ocean'] = yu_ocean.values
    salt_n = salt_i.roll(y_ocean=-1)
    salt_n.coords['y_ocean'] = yu_ocean.values
    # salt_ygrid will be on the vhrho grid:
    salt_ygrid = (salt_s + salt_n)/2

    # stack transports into 1d and drop any points not on contour:
    salt_xgrid = (salt_xgrid*dyt).where(mask_x_transport_numbered>0)
    salt_ygrid = (salt_ygrid*dxu).where(mask_y_transport_numbered>0)
    x_salt_1d = salt_xgrid.stack(contour_index = ['y_ocean', 'x_ocean'])
    y_salt_1d = salt_ygrid.stack(contour_index = ['y_ocean', 'x_ocean'])
    x_salt_1d = x_salt_1d.where(mask_x_numbered_1d>0,drop=True)
    y_salt_1d = y_salt_1d.where(mask_y_numbered_1d>0,drop=True)

    # combine all points on contour:
    salt_along_contour_i = xr.concat((x_salt_1d, y_salt_1d), dim = 'contour_index')
    salt_along_contour_i = salt_along_contour_i.sortby(contour_ordering)
    salt_along_contour_i.coords['contour_index'] = contour_index_array
    salt_along_contour_i = salt_along_contour_i.load()

    # write into larger array:
    salt_along_contour[:,:] = salt_along_contour_i

    del salt_i,salt_w,salt_e, salt_s, salt_n, salt_xgrid, salt_ygrid, x_salt_1d, y_salt_1d, salt_along_contour_i

    Form_contour = salt_along_contour

    salt = EPE.sel(yt_ocean = lat_range)
    if contour_dict[j_contour]['shifted']==True:
        salt = salt.roll(xt_ocean = 800)
        salt.coords['xt_ocean']=np.linspace(0.05,359.95,3600)

    # inititalise empty array
    salt_along_contour = xr.DataArray(np.zeros((len(salt.isopycnal_bins),len(contour_index_array))),
                                      coords = [salt.isopycnal_bins, contour_index_array],
                                      dims = ['isopycnal_bins', 'contour_index'],
                                      name = 'salt_along_contour')

    salt_i = salt.rename({'yt_ocean':'y_ocean', 'xt_ocean':'x_ocean'}) 

    # Note that this interpolation does not work as generically as e.g. salt.interp(),
    #    but it is much faster and doesn't require removing chunking (which also slow things down).
    # Be careful that your latitude range extends at least one point either direction beyond your contour.
    # If your domain is not the full longitude range, you will need to adapt this, so you have the correct interpolation
    #    only the edges of your domain (it assumes it is reentrant).
    # Need to overwrite coords, so these two variables can be added together:
    salt_w = salt_i.copy()
    salt_w.coords['x_ocean'] = xu_ocean.values
    salt_e = salt_i.roll(x_ocean=-1)
    salt_e.coords['x_ocean'] = xu_ocean.values
    # salt_xgrid will be on the uhrho grid:
    salt_xgrid = (salt_e + salt_w)/2

    salt_s = salt_i.copy()
    salt_s.coords['y_ocean'] = yu_ocean.values
    salt_n = salt_i.roll(y_ocean=-1)
    salt_n.coords['y_ocean'] = yu_ocean.values
    # salt_ygrid will be on the vhrho grid:
    salt_ygrid = (salt_s + salt_n)/2

    # stack transports into 1d and drop any points not on contour:
    salt_xgrid = (salt_xgrid*dyt).where(mask_x_transport_numbered>0)
    salt_ygrid = (salt_ygrid*dxu).where(mask_y_transport_numbered>0)
    x_salt_1d = salt_xgrid.stack(contour_index = ['y_ocean', 'x_ocean'])
    y_salt_1d = salt_ygrid.stack(contour_index = ['y_ocean', 'x_ocean'])
    x_salt_1d = x_salt_1d.where(mask_x_numbered_1d>0,drop=True)
    y_salt_1d = y_salt_1d.where(mask_y_numbered_1d>0,drop=True)

    # combine all points on contour:
    salt_along_contour_i = xr.concat((x_salt_1d, y_salt_1d), dim = 'contour_index')
    salt_along_contour_i = salt_along_contour_i.sortby(contour_ordering)
    salt_along_contour_i.coords['contour_index'] = contour_index_array
    salt_along_contour_i = salt_along_contour_i.load()

    # write into larger array:
    salt_along_contour[:,:] = salt_along_contour_i

    del salt_i,salt_w,salt_e, salt_s, salt_n, salt_xgrid, salt_ygrid, x_salt_1d, y_salt_1d, salt_along_contour_i

    EPE_contour = salt_along_contour

    salt = EKE.sel(yt_ocean = lat_range)
    if contour_dict[j_contour]['shifted']==True:
        salt = salt.roll(xt_ocean = 800)
        salt.coords['xt_ocean']=np.linspace(0.05,359.95,3600)

    # inititalise empty array
    salt_along_contour = xr.DataArray(np.zeros((len(salt.isopycnal_bins),len(contour_index_array))),
                                      coords = [salt.isopycnal_bins, contour_index_array],
                                      dims = ['isopycnal_bins', 'contour_index'],
                                      name = 'salt_along_contour')

    salt_i = salt.rename({'yt_ocean':'y_ocean', 'xt_ocean':'x_ocean'}) 

    # Note that this interpolation does not work as generically as e.g. salt.interp(),
    #    but it is much faster and doesn't require removing chunking (which also slow things down).
    # Be careful that your latitude range extends at least one point either direction beyond your contour.
    # If your domain is not the full longitude range, you will need to adapt this, so you have the correct interpolation
    #    only the edges of your domain (it assumes it is reentrant).
    # Need to overwrite coords, so these two variables can be added together:
    salt_w = salt_i.copy()
    salt_w.coords['x_ocean'] = xu_ocean.values
    salt_e = salt_i.roll(x_ocean=-1)
    salt_e.coords['x_ocean'] = xu_ocean.values
    # salt_xgrid will be on the uhrho grid:
    salt_xgrid = (salt_e + salt_w)/2

    salt_s = salt_i.copy()
    salt_s.coords['y_ocean'] = yu_ocean.values
    salt_n = salt_i.roll(y_ocean=-1)
    salt_n.coords['y_ocean'] = yu_ocean.values
    # salt_ygrid will be on the vhrho grid:
    salt_ygrid = (salt_s + salt_n)/2

    # stack transports into 1d and drop any points not on contour:
    salt_xgrid = (salt_xgrid*dyt).where(mask_x_transport_numbered>0)
    salt_ygrid = (salt_ygrid*dxu).where(mask_y_transport_numbered>0)
    x_salt_1d = salt_xgrid.stack(contour_index = ['y_ocean', 'x_ocean'])
    y_salt_1d = salt_ygrid.stack(contour_index = ['y_ocean', 'x_ocean'])
    x_salt_1d = x_salt_1d.where(mask_x_numbered_1d>0,drop=True)
    y_salt_1d = y_salt_1d.where(mask_y_numbered_1d>0,drop=True)

    # combine all points on contour:
    salt_along_contour_i = xr.concat((x_salt_1d, y_salt_1d), dim = 'contour_index')
    salt_along_contour_i = salt_along_contour_i.sortby(contour_ordering)
    salt_along_contour_i.coords['contour_index'] = contour_index_array
    salt_along_contour_i = salt_along_contour_i.load()

    # write into larger array:
    salt_along_contour[:,:] = salt_along_contour_i

    del salt_i,salt_w,salt_e, salt_s, salt_n, salt_xgrid, salt_ygrid, x_salt_1d, y_salt_1d, salt_along_contour_i

    EKE_contour = salt_along_contour

    
    save_dir = '/g/data/x77/cy8964/Post_Process/New_SO/'
    ds = xr.Dataset({'EKE_contour': EKE_contour,'Form_contour':Form_contour})
    ds.to_netcdf(save_dir+'SO_'+j_contour+'_contour_energies_10yr_smoothed_80km.nc')

    ds = xr.Dataset({'EPE_contour': EPE_contour})
    ds.to_netcdf(save_dir+'SO_'+j_contour+'_EPE_contour_smoothed_80km_fulldepth.nc')




A


distributed.utils_perf - WARNING - full garbage collections took 31% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 31% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 32% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 33% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 33% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 40% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 40% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 40% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 41% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took

P


distributed.utils_perf - WARNING - full garbage collections took 38% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 39% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 38% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 37% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 32% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 31% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 38% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 38% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 38% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took

B


...

distributed.utils_perf - WARNING - full garbage collections took 55% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 53% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 53% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 49% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 48% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 49% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 49% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 49% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 49% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took

Z8


distributed.utils_perf - WARNING - full garbage collections took 54% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 53% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 52% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 47% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 47% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 48% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 48% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 48% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 48% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took

Z9


distributed.utils_perf - WARNING - full garbage collections took 53% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 49% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 49% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 50% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 50% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 50% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 50% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 49% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 56% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took

#### for shifted data, want to move values on shifted grid back to original:

In [6]:
def find_end_index(j_contour):
    lat_along_contour = xr.open_dataset('/g/data/x77/cy8964/Post_Process/New_SO/SO_'+j_contour+'_latlon.nc').lat_along_contour
    lon_along_contour = xr.open_dataset('/g/data/x77/cy8964/Post_Process/New_SO/SO_'+j_contour+'_latlon.nc').lon_along_contour
    for i in np.arange(len(lon_along_contour)):
        if lon_along_contour[i]>80:
            lower_index = i
            break
    return lower_index

In [ ]:
for j in np.arange(8):
    print(j)
    j_contour = np.array(['O','Z3','Z4','Z5','Z6','Z7','Z8','Z9'])[j]
#     vh_eddy = xr.open_dataset('/g/data/x77/cy8964/Post_Process/New_SO/vh_eddy_SO_'+j_contour+'_1yr.nc').vh_eddy
#     h_bar_contour = xr.open_dataset('/g/data/x77/cy8964/Post_Process/New_SO/h_bar_contour_SO_'+j_contour+'.nc').h_bar
#     lat_along_contour = xr.open_dataset('/g/data/x77/cy8964/Post_Process/New_SO/SO_'+j_contour+'_latlon.nc').lat_along_contour
#     lon_along_contour = xr.open_dataset('/g/data/x77/cy8964/Post_Process/New_SO/SO_'+j_contour+'_latlon.nc').lon_along_contour
#    Form_contour = xr.open_dataset('/g/data/x77/cy8964/Post_Process/New_SO/SO_'+j_contour+'_contour_energies_10yr_smoothed_80km.nc').Form_contour
#    EKE_contour = xr.open_dataset('/g/data/x77/cy8964/Post_Process/New_SO/SO_'+j_contour+'_contour_energies_10yr_smoothed_80km.nc').EKE_contour
#     Reynolds_contour = xr.open_dataset('/g/data/x77/cy8964/Post_Process/New_SO/SO_'+j_contour+'_contour_energies_1yr.nc').Reynolds_contour
#     top_rho1_max_contour = xr.open_dataset('/g/data/x77/cy8964/Post_Process/New_SO/SO_'+j_contour+'bottom_min_top_max_rho1_contour.nc').top_rho1_max_contour
#     bottom_rho1_min_contour = xr.open_dataset('/g/data/x77/cy8964/Post_Process/New_SO/SO_'+j_contour+'bottom_min_top_max_rho1_contour.nc').bottom_rho1_min_contour
#     segment_length = xr.open_dataset('/g/data/x77/cy8964/Post_Process/New_SO/SO_'+j_contour+'_segment_length.nc').segment_length
#    Depth_gradient = xr.open_dataset('/g/data/x77/cy8964/Post_Process/New_SO/SO_'+j_contour+'_contour_depth_gradient_smoothed80km.nc').depth_grad_contour
#     EPE_contour = xr.open_dataset('/g/data/x77/cy8964/Post_Process/New_SO/SO_'+j_contour+'_EPE_contour_smoothed_80km.nc').EPE_contour
#     uh_bar = xr.open_dataset('/g/data/x77/cy8964/Post_Process/New_SO/SO_'+j_contour+'_mean_velocities_along_contours.nc').uh_bar_contour
#     vh_bar = xr.open_dataset('/g/data/x77/cy8964/Post_Process/New_SO/SO_'+j_contour+'_mean_velocities_along_contours.nc').vh_bar_contour
#     u_hat = xr.open_dataset('/g/data/x77/cy8964/Post_Process/New_SO/SO_'+j_contour+'_mean_velocities_along_contours.nc').u_hat_contour
#     v_hat = xr.open_dataset('/g/data/x77/cy8964/Post_Process/New_SO/SO_'+j_contour+'_mean_velocities_along_contours.nc').v_hat_contour
    T_eddy = xr.open_dataset('/g/data/x77/cy8964/Post_Process/New_SO/SO_'+j_contour+'_GL90_smoothed80km.nc').T_eddy

    
    
    lower_index = find_end_index(j_contour)
#     lon_along_contour_shiftedback = lon_along_contour.roll(contour_index = -lower_index, roll_coords = False)
#     for i in np.arange(len(lon_along_contour_shiftedback)):
#         if lon_along_contour_shiftedback[i] > 80:
#             lon_along_contour_shiftedback[i]=lon_along_contour_shiftedback[i]-360
#     lat_along_contour_shiftedback = lat_along_contour.roll(contour_index = -lower_index, roll_coords = False)
#     vh_eddy_shiftedback = vh_eddy.roll(contour_index = -lower_index, roll_coords = False)
#    Form_contour_shiftedback = Form_contour.roll(contour_index = -lower_index, roll_coords = False)
#    EKE_contour_shiftedback = EKE_contour.roll(contour_index = -lower_index, roll_coords = False)
#     Reynolds_contour_shiftedback = Reynolds_contour.roll(contour_index = -lower_index, roll_coords = False)
#     h_bar_shiftedback = h_bar_contour.roll(contour_index = -lower_index, roll_coords = False)
#     top_rho1_max_shiftedback = top_rho1_max_contour.roll(contour_index = -lower_index, roll_coords = False)
#     bottom_rho1_min_shiftedback = bottom_rho1_min_contour.roll(contour_index = -lower_index, roll_coords = False)
#     segment_length_shiftedback = segment_length.roll(contour_index = -lower_index, roll_coords = False)
#    Depth_gradient_shiftedback = Depth_gradient.roll(contour_index = -lower_index, roll_coords = False)
#    EPE_contour_shiftedback = EPE_contour.roll(contour_index = -lower_index, roll_coords = False)
#     uh_bar_shiftedback = uh_bar.roll(contour_index = -lower_index, roll_coords = False)
#     vh_bar_shiftedback = vh_bar.roll(contour_index = -lower_index, roll_coords = False)
#     u_hat_shiftedback = u_hat.roll(contour_index = -lower_index, roll_coords = False)
#     v_hat_shiftedback = v_hat.roll(contour_index = -lower_index, roll_coords = False)
    T_eddy_shiftedback = T_eddy.roll(contour_index = -lower_index, roll_coords = False)
    
    save_dir = '/g/data/x77/cy8964/Post_Process/New_SO/'
#     ds = xr.Dataset({'lat_along_contour': lat_along_contour_shiftedback,'lon_along_contour':lon_along_contour_shiftedback})
#     ds.to_netcdf(save_dir+'SO_'+j_contour+'_latlon_shiftedback.nc')

#     ds = xr.Dataset({'EKE_contour': EKE_contour_shiftedback,'Reynolds_contour':Reynolds_contour_shiftedback,'Form_contour':Form_contour_shiftedback})
#     ds.to_netcdf(save_dir+'SO_'+j_contour+'_contour_energies_shiftedback_1yr.nc')

#     ds = xr.Dataset({'vh_eddy': vh_eddy_shiftedback})
#     ds.to_netcdf(save_dir+'vh_eddy_SO_'+j_contour+'_shiftedback_1yr.nc')
    
#     ds = xr.Dataset({'h_bar': h_bar_shiftedback})
#     ds.to_netcdf(save_dir+'h_bar_contour_SO_'+j_contour+'_shiftedback.nc')
#     ds = xr.Dataset({'top_rho1_max_contour': top_rho1_max_shiftedback,'bottom_rho1_min_contour':bottom_rho1_min_shiftedback})
#     ds.to_netcdf(save_dir+'SO_'+j_contour+'bottom_min_top_max_rho1_contour_shiftedback.nc')
#     ds = xr.Dataset({'EPE_contour': EPE_contour_shiftedback})
#     ds.to_netcdf(save_dir+'SO_'+j_contour+'_EPE_contour_shiftedback.nc')
    #ds = xr.Dataset({'depth_grad_contour': Depth_gradient_shiftedback})
    #ds.to_netcdf(save_dir+'SO_'+j_contour+'_contour_depth_gradient_shiftedback.nc')
#     ds = xr.Dataset({'uh_bar_contour': uh_bar_shiftedback,'vh_bar_contour': vh_bar_shiftedback,
#                     'u_hat_contour': u_hat_shiftedback,'v_hat_contour': v_hat_shiftedback})
#     ds.to_netcdf(save_dir+'SO_'+j_contour+'_mean_velocities_along_contours_shiftedback.nc')
#     ds = xr.Dataset({'EKE_contour': EKE_contour_shiftedback,'Form_contour':Form_contour_shiftedback})
#     ds.to_netcdf(save_dir+'SO_'+j_contour+'_contour_energies_10yr_smoothed_80km_shiftedback.nc')

#     ds = xr.Dataset({'EPE_contour': EPE_contour_shiftedback})
#     ds.to_netcdf(save_dir+'SO_'+j_contour+'_EPE_contour_smoothed_80km_shiftedback.nc')

    ds = xr.Dataset({'T_eddy': T_eddy_shiftedback})
    ds.to_netcdf(save_dir+'SO_'+j_contour+'_GL90_smoothed80km_shiftedback.nc')

#     ds = xr.Dataset({'depth_grad_contour': Depth_gradient_shiftedback})
#     ds.to_netcdf(save_dir+'SO_'+j_contour+'_contour_depth_gradient_smoothed80km_shiftedback.nc')


0
1
2
3
4
5
6
7
